In [2]:
import sys 
import os
import shutil
import pandas as pd
import numpy as np
import SimpleITK as sitk
import numpy as np
val_tool_path = '/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/validation_tool/code/backend/flask_api'
sys.path.append(val_tool_path)
from ImageRegistration import ImgReg 
from SimMetrics import SimMet

In [24]:


# define paths and parameters
working_dir = "/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055/"
print_templates_path = os.path.join(working_dir,"meta","data")
print_templates = [root for root, dirs, files in os.walk(print_templates_path) for f in files if f.endswith('.dcm')]
print_templates = list(set(print_templates))
working_dir = "/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055/"
scans_high_dose_path = os.path.join(working_dir,"rawdata")
scans_high_dose = [root for root, dirs, files in os.walk(scans_high_dose_path) for f in files if f.endswith('.dcm')]
scans_high_dose = list(set(scans_high_dose))

# define parameters for mask generation
cutoff_window = (-200, 1500)
scale = 1
threshold_bg = -200
override = True
skip_index = 5
erosion = 2

In [25]:
print_templates
if '/home/uli/Insync/uligenske@gmail.com/Google Drive/Projekte_Uli/Deep_Learning/segmentation/projekte/phantom_datasets/231205_phantom247_055_unzipped_archive/meta/data/without_lesions/GAN_589_ALy055-1mm_LungTable_rem_5357_reg_scale079' in print_templates:
    print_templates.remove('/home/uli/Insync/uligenske@gmail.com/Google Drive/Projekte_Uli/Deep_Learning/segmentation/projekte/phantom_datasets/231205_phantom247_055_unzipped_archive/meta/data/without_lesions/GAN_589_ALy055-1mm_LungTable_rem_5357_reg_scale079')
print_templates

['/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055/meta/data/with_lesions/spheres/raw_589_ALy055-1mm_LungTable_rem_5357_reg_scale079_512x512x079',
 '/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055/meta/data/with_lesions/planePhantom/raw_Aly055_512x512x079_scale079_version3 (finale)',
 '/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055/meta/data/with_lesions/spheres/GAN_589_ALy055-1mm_LungTable_rem_5357_reg_scale079_512x512x079',
 '/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055/meta/data/with_lesions/planePhantom/GAN589_Aly055_512x512x079_scale079_version3 (finale)']

In [23]:
scans_high_dose

['/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055/rawdata/example_scans_CT4/86_120_700_FC08_AIDR3D_192726.253',
 '/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055/rawdata/example_scans_CT4/61_120_300_FC08_AIDR3D_180614.267',
 '/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055/rawdata/example_scans_CT4/90_120_700_BODY-SHARP_AICE_193137.193',
 '/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055/rawdata/example_scans_CT4/61_120_100_FC08_AIDR3D_171455.447',
 '/home/uli/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055/rawdata/example_scans_CT4/11_120_300_BODY-SHARP_AICE_180614.267',
 '/home/uli/

In [26]:
for print_template in print_templates:
    for scan_high_dose in scans_high_dose:
        # settings for CT scans
        config = {
            "preprocessing": "normalization",
            "model_net_metric": None
        }
        # define paths
        # dir name for analysis output
        analysis_name = f"{os.path.basename(print_template)}_{os.path.basename(scan_high_dose)}_{erosion}_{cutoff_window}_{skip_index}"
        analysis_name = os.path.join("sim_analyis_unzipped_archive/055", analysis_name)
        # create output folder
        if not os.path.exists(analysis_name):
            os.mkdir(analysis_name)
        elif override:
            shutil.rmtree(analysis_name)
            os.mkdir(analysis_name)
            
        # remove non dicom files from print template and scan high dose
        for f in os.listdir(print_template):
            if not f.lower().endswith(".dcm"):
                os.remove(os.path.join(print_template, f))
                
        for f in os.listdir(scan_high_dose):
            if not f.lower().endswith(".dcm"):
                os.remove(os.path.join(scan_high_dose, f))

        # I image registration
        image_reg_path = os.path.join(analysis_name, "1_image_registration")
        if not os.path.isdir(image_reg_path):
            os.mkdir(image_reg_path)
        elif override:
            shutil.rmtree(image_reg_path)
            os.mkdir(image_reg_path)

        reg = ImgReg(print_template, scan_high_dose, image_reg_path)
        reg.prepare_files()
        reg.register_images()
        reg.apply_trans_orig_vol()
        checkboard_image_path = reg.checkboard_plot(10)


        # II create mask and calc similarity metrics

        # import tensorflow as tf
        # inception = tf.keras.applications.inception_v3.InceptionV3(include_top=False, pooling="avg", input_shape=(512, 512, 3))
        scan_high_dose_reg = reg.reg_vol_dcm_path

        mask_dir = os.path.join(analysis_name, "2_mask")
        if not os.path.isdir(mask_dir):
            os.mkdir(mask_dir)
        else:
            shutil.rmtree(mask_dir)
            os.mkdir(mask_dir)

        sim = SimMet(print_template, scan_high_dose_reg, config, analysis_name, skip_index=skip_index)
        sim.calc_mask_image_1(threshold_bg=threshold_bg, threshold_within=cutoff_window, mask_path=mask_dir, fill_holes=True, erosion=erosion)
        sim.calc_mask_image_2(threshold_bg=threshold_bg, threshold_within=cutoff_window, mask_path=mask_dir, fill_holes=True, erosion=erosion)
        sim.calc_consensus_mask(mask_dir)
        #sim.calc_mask(threshold_bg=threshold_bg, threshold_within=cutoff_window, mask_path=mask_dir)
        # II calc similarity metrics
        # create mask and calc similarity metrics
        sim.calc_sim_metrics()
        sim_metrics = sim.get_sim_metrics()
        sim_metrics_df = pd.DataFrame(sim_metrics, index=[0])
        sim_metrics_df.to_csv(os.path.join(analysis_name, "sim_metrics.csv"), index=True)   
        # plot_dir = os.path.join(analysis_name, "plots")
        # if not os.path.isdir(plot_dir):
        #     os.mkdir(plot_dir)
        # else:
        #     shutil.rmtree(plot_dir)
        #     os.mkdir(plot_dir)

        # checkboard_image = sitk.ReadImage(checkboard_image_path)
        # checkboard_image_array = sitk.GetArrayFromImage(checkboard_image)
        # if skip_index > 0:
        #     checkboard_image_array = checkboard_image_array[skip_index:-skip_index,:,:]        
        # checkboard_image_array[sim.mask==0] = -1000
        # checkboard_image_array = sitk.GetImageFromArray(checkboard_image_array)
        # checkboard_image_array.SetSpacing(checkboard_image.GetSpacing())
        # checkboard_image_array.SetOrigin(checkboard_image.GetOrigin())
        # checkboard_image_array.SetDirection(checkboard_image.GetDirection())
    
        # sitk.WriteImage(checkboard_image_array, os.path.join(plot_dir, "checkboard_bg_removed.nii.gz"))

        # # III plot results
        # # ssim heatmap
        # sim.plot_ssim_heat_map()
    
        # # III plot results
        # #  substraction heatmap absolute
        # # substraction_path = os.path.join(plot_dir, "substractions")
        # # if not os.path.isdir(substraction_path):
        # #     os.mkdir(substraction_path)
        # # else:
        # #     shutil.rmtree(substraction_path)
        # #     os.mkdir(substraction_path)

        # sim.config["preprocessing"] = "no_preprocessing"
        # sim.calc_substract_image(absolute=True)
        # sim.plot_substract_heatmap()


In [1]:
# test of ssim scores 43
import sys 
import os
import shutil
import pandas as pd
import SimpleITK as sitk

val_tool_path = '/home/uli/data/Insync/uligenske@gmail.com/Google Drive/arbeit/validation_tool/code/backend/flask_api'
sys.path.append(val_tool_path)
from SimMetrics import SimMet, SimMetConfig
from ImageRegistration import ImgReg 
from piplineCombinations import run_reg_and_sim

image_1_dcm_path = '/home/uli/data/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/230915_phantom241_043_CT4/meta/data/without_lesions/old_gan_synth_Phm_origSize_20HU+_Intestin_Lung_Table_soft_211003_skaliert078'
image_2_dcm_path = "/home/uli/data/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/230915_phantom241_043_CT4/rawdata/rawdata_CT4/63_120_700_BODY-SHARP_AICE_192204.852"

output_folder = "ssim_tests"
cutoff_window = (-200, float("Inf"))

override = True

config_sim = {
    "preprocessing": "normalization",
    "skip_index": 5,
    "erosion": 2,
    "threshold_bg": -200
}

analysis_name = f"{os.path.basename(image_1_dcm_path)}_{os.path.basename(image_2_dcm_path)}_{config_sim['erosion']}_{cutoff_window}_{config_sim['skip_index']}"
analysis_name = os.path.join(output_folder, analysis_name)
# create output folder
if not os.path.exists(analysis_name):
    os.mkdir(analysis_name)
elif override:
    shutil.rmtree(analysis_name)
    os.mkdir(analysis_name)
            
# remove non dicom files from print template and scan high dose
for f in os.listdir(image_1_dcm_path):
    if not f.lower().endswith(".dcm"):
        os.remove(os.path.join(image_1_dcm_path, f))
                
for f in os.listdir(image_2_dcm_path):
    if not f.lower().endswith(".dcm"):
        os.remove(os.path.join(image_2_dcm_path, f))

# I image registration
image_reg_path = os.path.join(analysis_name, "1_image_registration")
if not os.path.isdir(image_reg_path):
    os.mkdir(image_reg_path)
elif override:
    shutil.rmtree(image_reg_path)
    os.mkdir(image_reg_path)

reg = ImgReg(image_1_dcm_path, image_2_dcm_path, image_reg_path)
reg.prepare_files()
reg.register_images()
reg.apply_trans_orig_vol()
checkboard_image_path = reg.checkboard_plot(10)


# II create mask and calc similarity metrics

# import tensorflow as tf
# inception = tf.keras.applications.inception_v3.InceptionV3(include_top=False, pooling="avg", input_shape=(512, 512, 3))
image_2_dcm_path_reg = reg.reg_vol_dcm_path


sim_analysis_path = os.path.join(analysis_name, "2_sim_analysis", str(cutoff_window))
configSim = SimMetConfig(preprocessing=config_sim["preprocessing"],
                          skip_index=config_sim["skip_index"],
                          erosion=config_sim["erosion"],
                          output_folder=sim_analysis_path)

sim = SimMet(image_1_dcm_path, image_2_dcm_path_reg, configSim)
sim.calc_mask_image_1(threshold_bg=config_sim["threshold_bg"], threshold_within=cutoff_window, save_mask=True, fill_holes=True)
sim.calc_mask_image_2(threshold_bg=config_sim["threshold_bg"], threshold_within=(-200, float("Inf")), save_mask=True, fill_holes=True)
sim.calc_consensus_mask()
#sim.calc_mask(threshold_bg=threshold_bg, threshold_within=cutoff_window, mask_path=mask_dir)
# II calc similarity metrics
# create mask and calc similarity metrics
sim.calc_sim_metrics()
sim_metrics = sim.get_sim_metrics()
sim_metrics_df = pd.DataFrame(sim_metrics, index=[0])
sim_metrics_df.to_csv(os.path.join(analysis_name, "sim_metrics.csv"), index=True)   
plot_dir = os.path.join(analysis_name, "plots")
if not os.path.isdir(plot_dir):
    os.mkdir(plot_dir)
else:
    shutil.rmtree(plot_dir)
    os.mkdir(plot_dir)

checkboard_image = sitk.ReadImage(checkboard_image_path)
checkboard_image_array = sitk.GetArrayFromImage(checkboard_image)
if config_sim["skip_index"] > 0:
        checkboard_image_array = checkboard_image_array[config_sim["skip_index"]:-config_sim["skip_index"],:,:]  
checkboard_image_array[sim.mask==0] = -1000
checkboard_image_array = sitk.GetImageFromArray(checkboard_image_array)
checkboard_image_array.SetSpacing(checkboard_image.GetSpacing())
checkboard_image_array.SetOrigin(checkboard_image.GetOrigin())
checkboard_image_array.SetDirection(checkboard_image.GetDirection())

sitk.WriteImage(checkboard_image_array, os.path.join(plot_dir, "checkboard_bg_removed.nii.gz"))


sim.plot_px_dist_input_images()
sim.plot_px_dist_preprocessed_images()

# III plot results
# ssim heatmap
sim.plot_ssim_heat_map()

sim.config.preprocessing = "no_preprocessing"
sim.calc_substract_image(absolute=True)
sim.plot_substract_heatmap()
sim.calc_substract_image(absolute=False)
sim.plot_substract_heatmap(zmin=-100, zmax=100, zmid=0, zstep=5, custColorscale=['blue','white','red'])
ssim_vals_043 = sim.metrics["ssim_array"]



In [2]:
# test of ssim scores 055
import sys 
import os
import shutil
import pandas as pd
import SimpleITK as sitk

val_tool_path = '/home/uli/data/Insync/uligenske@gmail.com/Google Drive/arbeit/validation_tool/code/backend/flask_api'
sys.path.append(val_tool_path)
from SimMetrics import SimMet, SimMetConfig
from ImageRegistration import ImgReg 
from piplineCombinations import run_reg_and_sim

image_1_dcm_path = "/home/uli/data/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055_CT4/meta/data/with_lesions/planePhantom/GAN589_Aly055_512x512x079_scale079_version3 (finale)"
image_2_dcm_path = "/home/uli/data/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/231205_phantom247_055_CT4/rawdata/example_scans_CT4/90_120_700_BODY-SHARP_AICE_193137.193"


output_folder = "ssim_tests"
cutoff_window = (-200, float("Inf"))

override = True

config_sim = {
    "preprocessing": "normalization",
    "skip_index": 5,
    "erosion": 2,
    "threshold_bg": -200
}

analysis_name = f"{os.path.basename(image_1_dcm_path)}_{os.path.basename(image_2_dcm_path)}_{config_sim['erosion']}_{cutoff_window}_{config_sim['skip_index']}"
analysis_name = os.path.join(output_folder, analysis_name)
# create output folder
if not os.path.exists(analysis_name):
    os.mkdir(analysis_name)
elif override:
    shutil.rmtree(analysis_name)
    os.mkdir(analysis_name)
            
# remove non dicom files from print template and scan high dose
for f in os.listdir(image_1_dcm_path):
    if not f.lower().endswith(".dcm"):
        os.remove(os.path.join(image_1_dcm_path, f))
                
for f in os.listdir(image_2_dcm_path):
    if not f.lower().endswith(".dcm"):
        os.remove(os.path.join(image_2_dcm_path, f))

# I image registration
image_reg_path = os.path.join(analysis_name, "1_image_registration")
if not os.path.isdir(image_reg_path):
    os.mkdir(image_reg_path)
elif override:
    shutil.rmtree(image_reg_path)
    os.mkdir(image_reg_path)

reg = ImgReg(image_1_dcm_path, image_2_dcm_path, image_reg_path)
reg.prepare_files()
reg.register_images()
reg.apply_trans_orig_vol()
checkboard_image_path = reg.checkboard_plot(10)


# II create mask and calc similarity metrics

# import tensorflow as tf
# inception = tf.keras.applications.inception_v3.InceptionV3(include_top=False, pooling="avg", input_shape=(512, 512, 3))
image_2_dcm_path_reg = reg.reg_vol_dcm_path


sim_analysis_path = os.path.join(analysis_name, "2_sim_analysis", str(cutoff_window))
configSim = SimMetConfig(preprocessing=config_sim["preprocessing"],
                          skip_index=config_sim["skip_index"],
                          erosion=config_sim["erosion"],
                          output_folder=sim_analysis_path)

sim = SimMet(image_1_dcm_path, image_2_dcm_path_reg, configSim)
sim.calc_mask_image_1(threshold_bg=config_sim["threshold_bg"], threshold_within=cutoff_window, save_mask=True, fill_holes=True)
sim.calc_mask_image_2(threshold_bg=config_sim["threshold_bg"], threshold_within=(-200, float("Inf")), save_mask=True, fill_holes=True)
sim.calc_consensus_mask()
#sim.calc_mask(threshold_bg=threshold_bg, threshold_within=cutoff_window, mask_path=mask_dir)
# II calc similarity metrics
# create mask and calc similarity metrics
sim.calc_sim_metrics()
sim_metrics = sim.get_sim_metrics()
sim_metrics_df = pd.DataFrame(sim_metrics, index=[0])
sim_metrics_df.to_csv(os.path.join(analysis_name, "sim_metrics.csv"), index=True)   
plot_dir = os.path.join(analysis_name, "plots")
if not os.path.isdir(plot_dir):
    os.mkdir(plot_dir)
else:
    shutil.rmtree(plot_dir)
    os.mkdir(plot_dir)

checkboard_image = sitk.ReadImage(checkboard_image_path)
checkboard_image_array = sitk.GetArrayFromImage(checkboard_image)
if config_sim["skip_index"] > 0:
        checkboard_image_array = checkboard_image_array[config_sim["skip_index"]:-config_sim["skip_index"],:,:]  
checkboard_image_array[sim.mask==0] = -1000
checkboard_image_array = sitk.GetImageFromArray(checkboard_image_array)
checkboard_image_array.SetSpacing(checkboard_image.GetSpacing())
checkboard_image_array.SetOrigin(checkboard_image.GetOrigin())
checkboard_image_array.SetDirection(checkboard_image.GetDirection())

sitk.WriteImage(checkboard_image_array, os.path.join(plot_dir, "checkboard_bg_removed.nii.gz"))


sim.plot_px_dist_input_images()
sim.plot_px_dist_preprocessed_images()

# III plot results
# ssim heatmap
sim.plot_ssim_heat_map()

sim.config.preprocessing = "no_preprocessing"
sim.calc_substract_image(absolute=True)
sim.plot_substract_heatmap()
sim.calc_substract_image(absolute=False)
sim.plot_substract_heatmap(zmin=-100, zmax=100, zmid=0, zstep=5, custColorscale=['blue','white','red'])
ssim_vals_055 = sim.metrics["ssim_array"]



In [3]:
import plotly.express as px

ssim_vals_df = pd.DataFrame({"ssim": ssim_vals_043 + ssim_vals_055, "phantom": ["043"]*len(ssim_vals_043) + ["055"]*len(ssim_vals_055) ,
                             "slice": list(range(len(ssim_vals_043))) + list(range(len(ssim_vals_055)))})

ssim_vals_df_summary = ssim_vals_df.groupby("phantom").agg({"ssim": ["mean", "std", "min", "max", "median", "count"]})
fig = px.scatter(data_frame=ssim_vals_df, x="slice", y="ssim", color="phantom",
                 title="SSIM Values", range_y=[0,1],
                 labels={"x": "Slice Index", "y": "SSIM"}, template="simple_white")
fig.show()


output_dir = "/home/uli/data/Insync/uligenske@gmail.com/Google Drive/arbeit/deep_learning/segmentation/datasets/phantom_datasets/plots/2) phantom similarity"
fig.write_html(os.path.join(output_dir, "ssim_values_vs_slices.html"))
ssim_vals_df.to_csv(os.path.join(output_dir, "ssim_values_vs_slices.csv"), index=False)

In [4]:
ssim_vals_df_summary

ssim                                              
             mean       std       min       max    median count
phantom                                                        
043      0.922964  0.008597  0.909349  0.944119  0.920128   182
055      0.930973  0.014318  0.894401  0.955577  0.930348   162